<a href="https://colab.research.google.com/github/princessivy/course/blob/main/model_dl_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Packages

In [2]:
!pip install ndjson --quiet
!pip install langdetect --quiet

     |████████████████████████████████| 981 kB 4.2 MB/s 


In [3]:
import pandas as pd
import ndjson
import gzip
from google.colab import drive
from pathlib import Path
from xgboost import XGBRegressor
from langdetect import detect

from sklearn.model_selection import cross_val_score


# Load Datasets

In [3]:
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
path_train = Path('/content/drive/MyDrive/Preprocessed_data/de eng_train_preprocessed/df_train_german.ndjson')

with open(path_train) as f:
    data = ndjson.load(f)
df_train = pd.DataFrame(data)

# Save Datasets

In [106]:
datasaver_to_ndjson(df_train_x, 'df_train_de_sentiment')

In [90]:
def datasaver_to_ndjson(df, name):
  # create flat list in dict form: {'col1': 'value', 'col2': 'value', ...} from df
  flat_list_back = []
  for i in range(len(df)):
    line = df.loc[i].to_dict()
    #line['industry'] = str(line['industry']) # use if idustry number (e.g. 13) should be enclosed in '' (e.g. '13')
    flat_list_back.append([line])

  filename = str(name) + '.ndjson'

  # https://stackoverflow.com/questions/21058935/python-json-loads-shows-valueerror-extra-data
  with open(filename, mode='w') as f:
    for item in flat_list_back:
      f.write('{}\n'.format(ndjson.dumps(item))) 

# Build Model

In [ ]:
hyperparameter tuning: gridsearch(!), randomised search
# https://www.kaggle.com/stuarthallows/using-xgboost-with-scikit-learn
# https://www.datacamp.com/community/tutorials/xgboost-in-python#apply

fastai
# https://fastai1.fast.ai/tabular.html

mega?
# https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794



# XGBoost

In [ ]:
import numpy as np

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

from sklearn.feature_extraction.text import TfidfVectorizer

# einlesen
from sklearn.compose import ColumnTransformer

In [150]:
X = df_train_x[['concatenated', 'sentiment_analysis']]
y = df_train_x['industry']

In [157]:
# wende tfidf an
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_x['concatenated'])

In [178]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=42)

In [160]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(train_X, train_y)

y_pred = xgb_model.predict(val_X)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored

In [183]:
# confusion matrix
print(confusion_matrix(y_pred, val_y))

[[ 19   0   1   0   0   0   0   0   0   0   0   0   1   0   0   1   0   0
    0]
 [  0  82   4   0   0   0   0   0   1   0   0   0   1   0   0   0   0   2
    0]
 [  5   3 241   5   2   2   1   1   7   3   3   2  10  36   2   4   3  18
    3]
 [  0   1   3 103   2   0   2   1   3   0   0   1   3  11   0   3   1   1
    0]
 [  2   0   2   1  56   0   0   0   0   1   4   4   0   1   0   6  17   0
    2]
 [  0   0   0   0   0  40   0   0   0   0   0   0   0   0   0   0   0   0
    0]
 [  0   0   0   0   0   3  56   0   0   0   0   1   3   0   0   7   0   0
    0]
 [  0   0   1   1   0   0   0  35   2   0   0   1   1   1   1   0   0   0
    0]
 [  0   1   3   0   0   0   0   2 172   2   0   0   4  10   1   0   0   0
    1]
 [  1   2   3   0   0   1   0   0   7 114   8   0   1   0   0   0   0   1
    0]
 [  2   1   3   0   3   0   0   0   1   6 128   0   0   1   0   7   7   1
    8]
 [  0   0   0   0   2   0   0   2   2   1   0  90   4   3   1   5  12   1
    0]
 [  1   0  26   4   0   3   

In [182]:
score = xgb_model.score(val_X, val_y)
score

0.7328098976921247

In [167]:
from sklearn import metrics
import numpy as np

In [179]:
logit_report = metrics.classification_report(val_y, y_pred, labels=val_y, output_dict=True)

In [181]:
pd.DataFrame(logit_report).T

,precision,recall,f1-score,support
43,0.877551,0.767857,0.819048,2.240000e+02
96,0.729389,0.860412,0.789501,8.740000e+02
11,0.686610,0.593596,0.636724,4.060000e+02
25,0.583333,0.368421,0.451613,1.520000e+02
116,0.924242,0.762500,0.835616,8.000000e+01
8,0.863636,0.351852,0.500000,5.400000e+01
135,0.648459,0.823843,0.725705,5.620000e+02
80,0.762980,0.761261,0.762120,4.440000e+02
48,0.761905,0.691892,0.725212,1.850000e+02
137,0.791209,0.610169,0.688995,1.180000e+02
